In [1]:
import logging
import os
import re
import sevenbridges as sbg

LOGGER = logging.getLogger(__name__)

In [2]:
def canonicalize_name(name: str) -> str:
    res = re.sub(r'\W', '_', name)
    res = ("_" + res) if re.match(r'^\d', res) else res
    return re.sub(r'_+', '_', res)

## Login to Seven Bridges

In [3]:
# The easiest way to setup login for testing purposes is to create a credentials file in ${HOME}/.sevenbridges/credentials
# [default]
# api_endpoint   = https://api.sbgenomics.com/v2/
# auth_token     = <TOKEN from https://igor.sbgenomics.com/developer/token>
# advance_access = false

api = sbg.Api()

In [4]:
# apps = api.apps.query("contextualize/sandbox-branden")
apps = api.apps.query("contextualize/sandbox-lindsey")
anapp = apps[0]
apps[:5]

[<App: id=contextualize/sandbox-lindsey/generate-random-forest-of-trees,
 <App: id=contextualize/sandbox-lindsey/digraph-synthetic-graph,
 <App: id=contextualize/sandbox-lindsey/db38pulling,
 <App: id=contextualize/sandbox-lindsey/find-files,
 <App: id=contextualize/sandbox-lindsey/create-custom-csv]

In [5]:
anapp._API

<API(https://api.sbgenomics.com/v2) - "c5b1b64dea6b4901ab4d9efca3577558">

## Generate Project Class

In [6]:
project_list = api.projects.query()
list(project_list)

[<Project: id=contextualize/debugging>,
 <Project: id=contextualize/utep-fvl>,
 <Project: id=contextualize/cwlcon2024-training>,
 <Project: id=contextualize/birdshot>,
 <Project: id=contextualize/sandbox-chen>,
 <Project: id=contextualize/onboarding-for-contextualize>,
 <Project: id=contextualize/sandbox-lindsey>,
 <Project: id=contextualize/sandbox-branden>]

In [7]:
# WARNING: Your mileage may vary -- this depends on your project access.
# I grabbed the penultimate project in the list because, as my sandbox,
# it was the one that was most familiar to me -- and if I screwed anything
# up during development I wouldn't break someone else's stuff. :-)

project = project_list[-2]
canonicalize_name(project.name)

'sandbox_lindsey'

## Executable Project

In [8]:
from inflection import camelize, underscore

In [9]:
# This was duplicated from above so the ExecutableProject code would exist in one place.
# I didn't duplicate import dependencies and, yes, I know that is wildly inconsistent.
def canonicalize_name(name: str) -> str:
    """
    Creates a valid python name from a string

    Convert a string to a canonical name by replacing non-word characters with underscores.
    If the string starts with a digit, an underscore is prepended to the string.

    Parameters
    ----------
    name: str
        The string to convert to a canonical name.

    Returns
    -------
    str
        The canonical name
    """
    res = re.sub(r'\W', '_', name)
    res = ("_" + res) if re.match(r'^\d', res) else res
    return re.sub(r'_+', '_', res)


def title(label: str) -> str:
    """
    Creates a title from a string

    Convert a string to a title by converting non-word characters to spaces
    and capitalizing the first letter of each word.

    Parameters
    ----------
    label: str
        The string to convert to a title.

    Returns
    -------
    str
        The title
    """
    import re
    res = re.sub(r'[^A-Za-z0-9]', ' ', label).strip().title()
    return res


def generator():
    """
    Generator that returns None forever. This is used in combination with
    tqdm to create a progress bar for a process that will take an
    indeterminate amount of time.

    Parameters
    ----------
    None

    Returns
    -------
    None
    """
    while True:
        yield


# Example Usage: ExecutableProject(project=project, cleanup=True)
# The goal behind this object is to read information from a Seven Bridges
# project and convert that project and its apps into python objects, that
# is, the project becomes a class and the apps become the member functions
# in that class.
class ExecutableProject:
    # Used to hold the project classes that have been created. (E.g.,
    # singleton construction.)
    _PROJECTS = dict()

    class ExecutableApp:
        def __init__(self, app: sbg.App, *,
                     cleanup: bool=False,
                     overwrite: bool=False,
                     polling_freq: float=10.0,):
            """
            Parameters
            ----------
            app: sbg.App
                The app to be converted into a function.
            cleanup: bool
                Whether to delete files generated on the remote server
                after they have been downloaded. Default: False
            overwrite: bool
                Whether to overwrite local files if they already exist.
                Default: False
            polling_freq: float
                The frequency at which to poll the app for completion.
                Default: 10.0 seconds. Minimum: 3.0 seconds.
            """
            self.app: sbg.App = app
            self.cleanup: bool = cleanup
            self.overwrite: bool = overwrite
            self._polling_freq: float = 10.0
            self.polling_freq = polling_freq

        @property
        def polling_freq(self) -> float:
            return self._polling_freq

        @polling_freq.setter
        def polling_freq(self, value: float) -> None:
            if value < 3.0:
                LOGGER.info("Minimum polling frequency: 3.0 sec")
            self._polling_freq = max(value, 3.0)

        @staticmethod
        def _parse_types(descr: None | str | list | dict) -> list[str]:
            """
            CWL is strongly typed. This parses the input/output types
            of the app into human-readable format.
            
            .. note::
                This are not guaranteed to map to python types -- some
                interpretation may be required on the part of the user.
                Obviously this is not perfect, but it is a start.

            Parameters
            ----------
            descr: None | str | list | dict
                The CWL description of the input or output type.

            Returns
            -------
            list[str]
                The types of the input or output.
            """
            if isinstance(descr, str):
                return [descr]
            elif isinstance(descr, list):
                if len(descr) == 0:
                    return []
                else:
                    types_ = []
                    for type_ in map(ExecutableProject.ExecutableApp._parse_types, descr):
                        types_.extend(type_)
                    return types_
            elif isinstance(descr, dict):
                if descr["type"] == "array":
                    return [f"list[{descr['items']}]" + \
                            ("?" if descr["items"].endswith("?") else "")]
                elif descr["type"] == "enum":
                    return "enum{%s}" % " | ".join(descr['symbols'])
                else:
                    raise TypeError(f"{descr['type']} is not recognized: {descr}")
            elif descr is None:
                return []
            else:
                raise TypeError(f"{type(descr)} is not recognized: {descr}")

        def _generate_docstring(self) -> str:
            """
            Generates a docstring for the app from the Seven Bridges
            App content.

            Returns
            -------
            str
                The docstring for the app.
            """
            app = self.app
            parse_types = ExecutableProject.ExecutableApp._parse_types
            doc = f"""
            {title(app.raw.get('label', 'Unnamed'))}
            {app.raw.get('doc', '')}
            """
            # add inputs docs
            inputs = app.raw.get("inputs", [])
            doc += f"""
            Parameters
            ----------
            """
            for input_ in inputs:
                types = [t.strip("?") for t in parse_types(input_["type"])]
                optional = any([t.endswith("?") for t in parse_types(input_["type"])])
                doc += f"""
                {input_['id']} : {'|'.join(types)}{' (optional)' if optional else ''}
                    {input_.get('label', 'No description provided')}. {input_.get('doc', '')}
                """
            doc += "\n"
            # add outputs docs
            outputs = app.raw.get("outputs", [])
            doc += f"""
            Returns
            -------
            """
            for output_ in outputs:
                types = [t.strip("?") for t in parse_types(output_["type"])]
                try:
                    doc += f"""
                    {output_['id']} : {'|'.join(types)}
                    """
                except Exception as e:
                    raise type(e)(f"Output: {output_}")
            return doc
        
        def generate_func(self):
            """
            Generates a function from the Seven Bridges App content.

            Returns
            -------
            function
                The function that calls the Seven Bridges App remotely.
            """
            import time
            from collections import deque
            from datetime import datetime
            from tqdm import tqdm

            parse_types = ExecutableProject.ExecutableApp._parse_types

            # local references to relevant variables.
            app = self.app
            cleanup = self.cleanup
            overwrite = self.overwrite
            polling_freq = self.polling_freq
            # build the function
            name = canonicalize_name(app.name)
            inputs = app.raw.get("inputs", [])
            outputs = app.raw.get("outputs", [])
            def func(self, **kwds):
                # check function call
                for input_ in inputs:
                    key = input_["id"]
                    optional = any([t.endswith("?") for t in parse_types(input_["type"])])
                    if not optional and key not in kwds:
                        raise ValueError(f"{key!r} missing in {name!r}")
                # TODO: SUPPORT INPUT FILE UPLOAD
                # CALL THE FUNCTION
                api = app._API
                task_name = name + "-" + datetime.now().strftime("%Y-%m-%d-%H:%M:%S")
                project_name = app.project
                app_name = f"{project_name}/{app.name}"
                try:
                    # Create the Seven Bridges task and, because run = True, start the task.
                    task = api.tasks.create(
                        name=task_name,
                        project=project_name,
                        app=app_name,
                        inputs=kwds,
                        run=True)
                    LOGGER.info(f"Task {task.id} has been submitted. This may take a while.")
                    # Track progress
                    pbar = tqdm(generator())
                    for _ in pbar:
                        pbar.set_description(f"Task {task.id!r}: {task.status}")
                        task.reload()
                        if task.status not in ("QUEUED", "RUNNING"):
                            break
                        time.sleep(polling_freq)
                except sbg.errors.SbgError:
                    print(f"Unable to run {name!r}")
                    raise
                except:
                    # Abort the task on error
                    print(f"Aborting task {task.id!r}")
                    try:
                        task.abort()
                    except:
                        # Abort typically fails because there is no task to abort.
                        # This except assumes that because I haven't looked into all
                        # the possible failure modes.
                        pass
                    raise
                else:
                    # PROCESS THE RESULTS
                    # task.outputs.values() does not work as expected.
                    LOGGER.info(f"Task {task.id} has finished. Any resulting files are being downloaded.")
                    files = [v for k,v in task.outputs.items() if isinstance(v, sbg.File)]
                    downloads = deque()
                    for file_ in files:
                        # start all files downloading...
                        LOGGER.debug(f"Downloading to {file_.name}")
                        download = file_.download(file_.name, overwrite=overwrite, wait=False)
                        downloads.append(download)
                        download.start()
                    for download in downloads:
                        # wait for downloads...
                        download.wait()
                    if cleanup:
                        LOGGER.info("Removing remote files.")
                        for file_ in files:
                            # cleanup
                            LOGGER.debug(f"Deleting {file_.name} ({file_.id}) from Seven Bridges.")
                            file_.delete()
                return task
            # update the function
            # There may be a few other things to do here to ensure the function
            # is properly documented, but this is a start.
            # func.__module__ = __name__  # This may be useful at implementation.
            func.__name__ = name
            func.__doc__ = self._generate_docstring()
            # finished
            return func
            # setattr(type(self), name, func)
    
    def __new__(cls, *,
                project: sbg.Project,
                cleanup: bool=False,
                overwrite: bool=False,
                polling_freq: float=10.0,):
        """
        This is the constructor for the ExecutableProject class. It
        generates a project class that will contain executable apps
        as methods.

        Why overload __new__? Because __new__ controls object creation
        (versus __init__, which controls object initialization) and
        unlike more typical behavior, this does not create an instance
        of the enclosing class. This creates an instance of the
        project class.
        """
        name = canonicalize_name(project.name)
        classname = camelize(name)
        if classname in cls._PROJECTS:
            # class has already been constructed for this project
            type_ = cls._PROJECTS[classname]
        else:
            # generate a class for this project
            LOGGER.debug(f"Generating class for {classname}")
            # Project class __init__ function.
            # TODO: I would rather represent this as a string and inject it into the namespace
            # using 'exec'. I tried that, but was unsuccessful -- I just couldn't get the syntax
            # right. That would remove the annotation warning that pop up below.
            def init(self, *,
                project: sbg.Project,
                cleanup: bool=False,
                overwrite: bool=False,
                polling_freq: float=10.0,
            ):
                """
                Project classes contain method functions that call Seven
                Bridges Apps. This constructor initializes the project class.

                Parameters
                ----------
                project: sbg.Project
                    The project to be converted into a class.
                cleanup: bool
                    Whether to delete files generated on the remote server
                    after they have been downloaded. Default: False
                overwrite: bool
                    Whether to overwrite local files if they already exist.
                    Default: False
                polling_freq: float
                    The frequency at which to poll the app for completion.
                    Default: 10.0 seconds. Minimum: 3.0 seconds.
                """
                super(type(self), self).__init__()
                self.project: sbg.Project = project
                self.cleanup: bool = cleanup
                self.overwrite: bool = overwrite
                self.polling_freq: float = polling_freq
            namespace = {"__init__": init}
            for app in project.get_apps():
                LOGGER.debug(f"Creating executable for {app.name}")
                func = cls.ExecutableApp(app,
                                         cleanup=cleanup,
                                         overwrite=overwrite,
                                         polling_freq=polling_freq,
                                        ).generate_func()
                namespace[func.__name__] = func
            type_ = type(
                classname,
                (),
                namespace,)
            # Singleton pattern to avoid recreating this class definition
            # every time someone requests this object.
            cls._PROJECTS[classname] = type_
        return type_(project=project,
                     cleanup=cleanup,
                     overwrite=overwrite,
                     polling_freq=polling_freq,)

    @classmethod
    def reset(cls, name: None | str | list[str]=None):
        LOGGER.debug("Resetting executable project definitions.")
        if name is None:
            del cls._PROJECTS
            cls._PROJECTS = dict()
        else:
            for key in [name] if isinstance(name, str) else name:
                if key in cls._PROJECTS:
                    del cls._PROJECTS[key]

### Individual Tests

In [10]:
LOGGER.setLevel(logging.DEBUG)
# sandbox_branden = ExecutableProject(project=project, cleanup=True)
sandbox_lindsey = ExecutableProject(project=project, cleanup=True)

DEBUG:__main__:Generating class for SandboxLindsey
DEBUG:__main__:Creating executable for generate-random-forest-of-trees
DEBUG:__main__:Creating executable for digraph-synthetic-graph
DEBUG:__main__:Creating executable for db38pulling
DEBUG:__main__:Creating executable for find-files
DEBUG:__main__:Creating executable for create-custom-csv
DEBUG:__main__:Creating executable for dummy-enum
DEBUG:__main__:Creating executable for test-expansion
DEBUG:__main__:Creating executable for test-birdshot
DEBUG:__main__:Creating executable for label-okuma-runs
DEBUG:__main__:Creating executable for query-db41
DEBUG:__main__:Creating executable for okuma-runid
DEBUG:__main__:Creating executable for gtadexp-exploration
DEBUG:__main__:Creating executable for filter-dataset
DEBUG:__main__:Creating executable for delme
DEBUG:__main__:Creating executable for gtadexp-histograms
DEBUG:__main__:Creating executable for plot-histogram
DEBUG:__main__:Creating executable for driveam-graph-to-csv
DEBUG:__main_

In [11]:
sandbox_lindsey.digraph_synthetic_graph(url="https://httpbin.org/get")

INFO:__main__:Task 47145c02-e17c-47fb-be3c-0d93464eb413 has been submitted. This may take a while.
Task '47145c02-e17c-47fb-be3c-0d93464eb413': RUNNING: : 13it [02:11, 10.12s/it]
INFO:__main__:Task 47145c02-e17c-47fb-be3c-0d93464eb413 has finished. Any resulting files are being downloaded.
DEBUG:__main__:Downloading to forest-2024-10-17T135544.json
INFO:__main__:Removing remote files.
DEBUG:__main__:Deleting forest-2024-10-17T135544.json (6711177cb70b532f9ba058f1) from Seven Bridges.


<Task: id=47145c02-e17c-47fb-be3c-0d93464eb413>

In [13]:
sandbox_lindsey.generate_random_forest_of_trees(save_image=True)

INFO:__main__:Task a20ea57c-8059-4436-8c55-900452d9f794 has been submitted. This may take a while.
Task 'a20ea57c-8059-4436-8c55-900452d9f794': RUNNING: : 11it [01:51, 10.17s/it]
INFO:__main__:Task a20ea57c-8059-4436-8c55-900452d9f794 has finished. Any resulting files are being downloaded.
DEBUG:__main__:Downloading to forest-2024-10-17T135836.png
DEBUG:__main__:Downloading to forest-2024-10-17T135836.json
INFO:__main__:Removing remote files.
DEBUG:__main__:Deleting forest-2024-10-17T135836.png (67111827fa45275d09efa7dc) from Seven Bridges.
DEBUG:__main__:Deleting forest-2024-10-17T135836.json (67111827fa45275d09efa7de) from Seven Bridges.


<Task: id=a20ea57c-8059-4436-8c55-900452d9f794>

### Test All Projects

In [13]:
LOGGER.setLevel(logging.DEBUG)

ExecutableProject.reset()

projects = {
    canonicalize_name(p.name):ExecutableProject(project=p, cleanup=True)
    for p in api.projects.query().all()
}

DEBUG:__main__:Resetting executable project definitions.
DEBUG:__main__:Generating class for UtepFvl
DEBUG:__main__:Generating class for Cwlcon2024Training
DEBUG:__main__:Generating class for Birdshot
DEBUG:__main__:Generating class for SandboxChen
DEBUG:__main__:Creating executable for db38pulling
DEBUG:__main__:Creating executable for sandbox
DEBUG:__main__:Creating executable for command-line
DEBUG:__main__:Creating executable for dsc-report-creator
DEBUG:__main__:Creating executable for dsc-report-creator
DEBUG:__main__:Creating executable for dsc-analysis
DEBUG:__main__:Creating executable for aim-dsc-analysis
DEBUG:__main__:Creating executable for dsc-file-reader
DEBUG:__main__:Creating executable for dsc-data-analyzer
DEBUG:__main__:Creating executable for dsc-analysis
DEBUG:__main__:Creating executable for excel2json
DEBUG:__main__:Creating executable for excel2json
DEBUG:__main__:Creating executable for excel2json
DEBUG:__main__:Creating executable for cc-hello-world
DEBUG:__m

In [14]:
projects

{'utep_fvl': <__main__.UtepFvl at 0x11029af30>,
 'cwlcon2024_training': <__main__.Cwlcon2024Training at 0x107bcb4a0>,
 'birdshot': <__main__.Birdshot at 0x107ec7260>,
 'sandbox_chen': <__main__.SandboxChen at 0x107e9bbf0>,
 'Onboarding_for_Contextualize': <__main__.OnboardingForContextualize at 0x107ffc920>,
 'sandbox_lindsey': <__main__.SandboxLindsey at 0x107fb6570>,
 'sandbox_branden': <__main__.SandboxBranden at 0x1106045f0>}

In [15]:
p = projects["sandbox_lindsey"]

In [16]:
[_ for _ in dir(p) if not _.startswith("_")]

['calc',
 'cleanup',
 'concat_xlsx',
 'create_custom_csv',
 'db38pulling',
 'delme',
 'download_file',
 'driveam_graph_to_csv',
 'dummy_enum',
 'filter_dataset',
 'find_files',
 'generate_ni_srjt_summary',
 'grab_from_json',
 'gtadexp_exploration',
 'gtadexp_histograms',
 'jq',
 'label_okuma_runs',
 'ni_srjt_xlsx_processor',
 'okuma_runid',
 'overwrite',
 'plot',
 'plot_histogram',
 'polling_freq',
 'project',
 'query_db41',
 'scatter_test',
 'sed',
 'select_column',
 'srjt_json_excel_pairs',
 'test_birdshot',
 'test_birdshot_integration',
 'test_expansion',
 'test_pd']

## Executable Project Rewrite

In [ ]:
def canonicalize_name(name: str) -> str:
    """
    Creates a valid python name from a string

    Convert a string to a canonical name by replacing non-word characters with underscores.
    If the string starts with a digit, an underscore is prepended to the string.

    Parameters
    ----------
    name: str
        The string to convert to a canonical name.

    Returns
    -------
    str
        The canonical name
    """
    res = re.sub(r'\W', '_', name)
    res = ("_" + res) if re.match(r'^\d', res) else res
    return re.sub(r'_+', '_', res)


def title(label: str) -> str:
    """
    Creates a title from a string

    Convert a string to a title by converting non-word characters to spaces
    and capitalizing the first letter of each word.

    Parameters
    ----------
    label: str
        The string to convert to a title.

    Returns
    -------
    str
        The title
    """
    import re
    res = re.sub(r'[^A-Za-z0-9]', ' ', label).strip().title()
    return res


def generator():
    """
    Generator that returns None forever. This is used in combination with
    tqdm to create a progress bar for a process that will take an
    indeterminate amount of time.

    Parameters
    ----------
    None

    Returns
    -------
    None
    """
    while True:
        yield


class Type:
    def __init__(self, description: str):
        self.description = description
        self.allowed = []

In [ ]:
from contextlib import contextmanager
from enum import Enum
from pathlib import Path
from petname import generate as random_name


class File:
    def __init__(self, file: None | sbg.File=NotImplementedError, *, local: None | str | Path=None):
        self._remote = file
        self._local = local

    @property
    def local_path(self):
        return self._local or getattr(self._file, "name")
    
    @property
    def file(self) -> None | sbg.File:
        return self._remote
    
    @file.setter
    def file(self, value: sbg.File):
        self._remote = value
    
    @contextmanager
    def open(self, mode='r'):
        if self.local_path is None:
            raise ValueError("No local path set.")
        resource = open(self.local_path, mode)
        try:
            yield resource
        finally:
            resource.close()

    # proxy methods
    def __getattr__(self, name):
        attr = getattr(self, name)
        if attr is None:
            if self._remote is None:
                raise ValueError("No file set.")
            return getattr(self._remote, name)

    def download(self, *args, **kwds):
        if self._remote is None:
            raise ValueError("No file set.")
        opts = {k:v for k,v in zip(["path",
                                    "retry",
                                    "timeout",
                                    "chunk_size",
                                    "wait",
                                    "overwrite",],
                                    args)}
        opts = opts.update(kwds)
        opts["path"] = opts.get("path", self.local_path)
        self._local = opts["path"]
        if self.local_path is None:
            raise ValueError("No local path set.")
        self._remote.download(**opts)

    def upload(self, *args, **kwds):
        opts = {k:v for k,v in zip(["path",
                                    "project",
                                    "parent",
                                    "file_name",
                                    "overwrite",
                                    "retry",
                                    "timeout",
                                    "part_size",
                                    "wait",
                                    "api",],
                                    args)}
        opts = opts.update(kwds)
        opts["path"] = opts.get("path", self.local_path)
        opts["wait"] = opts.get("wait", True)
        upload = self._remote.upload(**opts)
        # set the remote file object
        if opts["wait"]:
            self._remote = upload.result()
        self._local = opts["path"]


class Type:
    TYPES = {
        "string": str,
        "boolean": bool,
        "int": int,
        "long": int,
        "float": float,
        "double": float,
        "null": type(None),
        "array": list,
        "record": dict,
        "File": File
    }

    def __init__(self, description: None | str | list | dict):
        def get_type(value) -> list[type]:
            if isinstance(value, str):
                # named type
                return [self.TYPES[value]]
            elif isinstance(value, list):
                # list of possible values
                return [get_type(v) for v in value]
            elif isinstance(value, tuple):
                # enumeration
                if value not in cls.TYPES:
                    cls.TYPES[value] = Enum(
                        camelize(random_name(separator="_") + "Enum"),
                        {k:v for v,k in enumerate(value)})
                return [cls.TYPES[value]]
            elif isinstance(value, dict):
                # complex types (array or enum)
                type_ = value["type"]
                return [{
                    "array": lambda: get_type(type_),
                    "enum": lambda: get_type(tuple(value["symbols"]))
                }[type_]()]
            elif value is None:
                # null type
                return [cls.TYPES["null"]]
            else:
                raise TypeError(f"Invalid type description: {value}")
            
        cls = self.__class__
        self.description = description
        self.allowed = get_type(description)
    
    def __call__(self):
        pass

In [ ]:
class Type:
    TYPES = {
        "string": str,
        "boolean": bool,
        "int": int,
        "long": int,
        "float": float,
        "double": float,
        "null": type(None),
        "array": list,
        "record": dict,
    }

    def __init__(self, description: None | str | list | dict):
        def get_type(value) -> list[type]:
            if isinstance(value, str):
                # named type
                return [self.TYPES[value]]
            elif isinstance(value, list):
                # list of possible values
                return [get_type(v) for v in value]
            elif isinstance(value, tuple):
                # enumeration
                if value not in cls.TYPES:
                    cls.TYPES[value] = Enum(
                        camelize(random_name(separator="_") + "Enum"),
                        {k:v for v,k in enumerate(value)})
                return [cls.TYPES[value]]
            elif isinstance(value, dict):
                # complex types (array or enum)
                type_ = value["type"]
                return [{
                    "array": lambda: get_type(type_),
                    "enum": lambda: get_type(tuple(value["symbols"]))
                }[type_]()]
            elif value is None:
                # null type
                return [cls.TYPES["null"]]
            else:
                raise TypeError(f"Invalid type description: {value}")
    
        cls = type(self)
        self.description = description
        self.allowed = get_type(description)
    
    def __call__(self):
        pass